In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Google_ML_Camp

/content/drive/MyDrive/Google_ML_Camp


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pwd

/content/drive/MyDrive/Google_ML_Camp


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.5/870.5 kB 15.5 MB/s eta 0:00:00


# roboflow api (png file)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("jyyun").project("dmg_dup3")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...


- epch = 200
- png file (roboflow api)
- optimizer='Adam'

# png 파일

In [ ]:
# Colab에 Ultralytics 설치
!pip install ultralytics

# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
from torchvision.ops import DeformConv2d
from ultralytics import YOLO

# Deformable Convolution 모듈 정의
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        super(DeformableConv2d, self).__init__()
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, stride=stride, padding=padding)
        self.deform_conv = DeformConv2d(in_channels, out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, bias=False)

    def forward(self, x):
        offset = self.offset_conv(x)
        out = self.deform_conv(x, offset)
        return out

# DFPN 모듈 정의
class DFPN(nn.Module):
    def __init__(self, in_channels_list, out_channels):
        super(DFPN, self).__init__()
        self.convs = nn.ModuleList()
        for in_channels in in_channels_list:
            self.convs.append(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0))

    def forward(self, features):
        out_features = []
        for i, feature in enumerate(features):
            out = self.convs[i](feature)
            if i > 0:
                out = out + out_features[i - 1]  # 이전 레벨의 특징과 결합
            out_features.append(out)
        return out_features

# Custom YOLOv8 모델 정의
class CustomYOLOv8Seg(YOLO):
    def __init__(self, model_path='yolov8n-seg.pt'):
        super().__init__(model_path)
        # 기존 백본 교체
        self.model.model[0] = CustomBackbone()  # YOLOv8의 첫 번째 계층이 백본임
        self.model.model[1] = DFPN([64, 128, 256], 256)  # YOLOv8의 두 번째 계층이 넥임

    def forward(self, x):
        return self.model(x)

# 수정된 백본 정의
class CustomBackbone(nn.Module):
    def __init__(self):
        super(CustomBackbone, self).__init__()
        # 간단한 Custom Backbone 정의
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.deform_conv = DeformableConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.deform_conv(x)
        x = self.conv2(x)
        return x

# 커스텀 모델 학습 및 평가 함수
def train_custom_model():
    # 모델 인스턴스 생성
    model = CustomYOLOv8Seg()

    # 데이터셋 경로 설정
    dataset_path = '/content/drive/MyDrive/Google_ML_Camp/dmg_dup3.v1-dmg_dup3.yolov8/data.yaml'

    # 모델 학습
    model.train(data=dataset_path, epochs=200, imgsz=640, optimizer='Adam')

    # 모델 평가
    model.val(data=dataset_path)
7
train_custom_model()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/Google_ML_Camp/dmg_dup3.v1-dmg_dup3.yolov8/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split

100%|██████████| 755k/755k [00:00<00:00, 15.2MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/Google_ML_Camp/dmg_dup3.v1-dmg_dup3.yolov8/train/labels... 941 images, 0 backgrounds, 0 corrupt: 100%|██████████| 941/941 [00:37<00:00, 25.05it/s]


train: New cache created: /content/drive/MyDrive/Google_ML_Camp/dmg_dup3.v1-dmg_dup3.yolov8/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


200 epochs completed in 0.532 hours.
Optimizer stripped from runs/segment/train15/weights/last.pt, 6.2MB
Optimizer stripped from runs/segment/train15/weights/best.pt, 6.2MB

Validating runs/segment/train15/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n-seg summary (fused): 213 layers, 2,938,539 parameters, 0 gradients, 10.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.65it/s]


                   all         23         51      0.364      0.382       0.34      0.227      0.727      0.241      0.356      0.229
           back_ripped          5          6     0.0525      0.167      0.205      0.124      0.758      0.167      0.198     0.0952
             back_wear          6         11      0.151      0.182      0.139     0.0437          0          0      0.108     0.0327
          front_folded          5          7       0.46      0.143      0.165      0.107          1          0    0.00177   0.000709
          front_ripped          4          4          1          0    0.00542    0.00332          1          0     0.0831    0.00996
            front_wear          5          9      0.316      0.111      0.114     0.0332          1          0      0.231      0.131
           side_ripped          2          2      0.151          1      0.995      0.895       0.59          1      0.995      0.821
             side_wear          3          3      0.409      0.333   

val: Scanning /content/drive/MyDrive/Google_ML_Camp/dmg_dup3.v1-dmg_dup3.yolov8/valid/labels.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.07it/s]


                   all         23         51      0.365      0.382      0.342      0.232      0.729      0.241      0.348      0.228
           back_ripped          5          6     0.0525      0.167      0.204      0.124      0.758      0.167      0.198     0.0952
             back_wear          6         11      0.152      0.182      0.148     0.0437          0          0     0.0895     0.0324
          front_folded          5          7      0.461      0.143      0.164      0.107          1          0    0.00177   0.000708
          front_ripped          4          4          1          0    0.00535    0.00329          1          0     0.0321    0.00484
            front_wear          5          9      0.315      0.111      0.114     0.0372          1          0      0.231      0.124
           side_ripped          2          2      0.163          1      0.995      0.895      0.601          1      0.995      0.821
             side_wear          3          3      0.408      0.333   

# without Adam

In [ ]:
# Colab에 Ultralytics 설치
!pip install ultralytics
!pip install roboflow

# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
from torchvision.ops import DeformConv2d
from ultralytics import YOLO
from roboflow import Roboflow

# Roboflow API로 데이터셋 다운로드
rf = Roboflow(api_key="Yb6O8aadRMORAdfsReiF")
project = rf.workspace("jyyun").project("dmg_dup3")
version = project.version(1)
dataset = version.download("yolov8")

# Deformable Convolution 모듈 정의
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        super(DeformableConv2d, self).__init__()
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, stride=stride, padding=padding)
        self.deform_conv = DeformConv2d(in_channels, out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, bias=False)

    def forward(self, x):
        offset = self.offset_conv(x)
        out = self.deform_conv(x, offset)
        return out

# DFPN 모듈 정의
class DFPN(nn.Module):
    def __init__(self, in_channels_list, out_channels):
        super(DFPN, self).__init__()
        self.convs = nn.ModuleList()
        for in_channels in in_channels_list:
            self.convs.append(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0))

    def forward(self, features):
        out_features = []
        for i, feature in enumerate(features):
            out = self.convs[i](feature)
            if i > 0:
                out = out + out_features[i - 1]  # 이전 레벨의 특징과 결합
            out_features.append(out)
        return out_features

# Custom YOLOv8 모델 정의
class CustomYOLOv8Seg(YOLO):
    def __init__(self, model_path='yolov8n-seg.pt'):
        super().__init__(model_path)
        # 기존 백본 교체
        self.model.model[0] = CustomBackbone()  # YOLOv8의 첫 번째 계층이 백본임
        self.model.model[1] = DFPN([64, 128, 256], 256)  # YOLOv8의 두 번째 계층이 넥임

    def forward(self, x):
        return self.model(x)

# 수정된 백본 정의
class CustomBackbone(nn.Module):
    def __init__(self):
        super(CustomBackbone, self).__init__()
        # 간단한 Custom Backbone 정의
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.deform_conv = DeformableConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.deform_conv(x)
        x = self.conv2(x)
        return x

# 커스텀 모델 학습 및 평가 함수
def train_custom_model():
    # 모델 인스턴스 생성
    model = CustomYOLOv8Seg()

    # Roboflow에서 다운로드된 데이터셋 경로 설정
    dataset_path = f'{dataset.location}/data.yaml'

    # 모델 학습
    model.train(data=dataset_path, epochs=200, imgsz=640)

    # 모델 평가
    model.val(data=dataset_path)

train_custom_model()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.5/870.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dmg_dup3-1 in yolov8:: 100%|██████████| 1950/1950 [00:00<00:00, 7501.05it/s]


100%|██████████| 6.74M/6.74M [00:00<00:00, 83.5MB/s]


Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/dmg_dup3-1/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]


...

                   all         23         51      0.371      0.462      0.421      0.236      0.334      0.431      0.378       0.23

200 epochs completed in 0.490 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/segment/train/weights/best.pt, 6.2MB

Validating runs/segment/train/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n-seg summary (fused): 213 layers, 2,938,539 parameters, 0 gradients, 10.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.90it/s]


                   all         23         51      0.725      0.391      0.448      0.298      0.669      0.365      0.418      0.283
           back_ripped          5          6      0.319      0.167     0.0998     0.0479      0.319      0.167      0.128     0.0556
             back_wear          6         11      0.432      0.182      0.268     0.0908      0.215     0.0909      0.159     0.0444
          front_folded          5          7      0.566      0.286       0.28     0.0857      0.283      0.143     0.0658     0.0101
          front_ripped          4          4          1          0          0          0          1          0          0          0
            front_wear          5          9          1          0      0.306     0.0959          1          0      0.307      0.131
           side_ripped          2          2      0.294        0.5      0.538      0.527      0.293        0.5      0.566      0.492
             side_wear          3          3          1       0.66   

val: Scanning /content/dmg_dup3-1/valid/labels.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.65it/s]


                   all         23         51      0.724      0.391      0.452      0.302      0.668      0.366      0.419      0.283
           back_ripped          5          6      0.319      0.167        0.1      0.048      0.319      0.167      0.128     0.0556
             back_wear          6         11      0.429      0.182      0.269     0.0911      0.214     0.0909       0.16     0.0438
          front_folded          5          7      0.566      0.286       0.28      0.086      0.283      0.143     0.0664     0.0103
          front_ripped          4          4          1          0          0          0          1          0          0          0
            front_wear          5          9          1          0      0.306     0.0961          1          0      0.306      0.131
           side_ripped          2          2      0.293        0.5      0.572       0.53      0.293        0.5      0.572      0.494
             side_wear          3          3          1      0.661   